In [2]:
# importing libraries
from utils import (
     prompts,
     llm,    
)
from core import hybrid_search
from utils.helpers import format_search_results, re_rank_docs
from ddgs import DDGS 
from IPython.display import Markdown, display

In [ ]:
# Extracting relevant documents from the vector database 
# Note: we are using qdrant vector db, which we already created for our UAE Legal RAG usecase
user_query = "what are the various types of recruitment pattern in uae?"
retrivedDocs = hybrid_search.retreive_relevant_docs(user_query)

In [17]:
# we will make use of decision prompt to decide if we have relevant context or not
# it returns 'Yes' if the context has an answer else it returns 'No' as answer
decision_prompt = prompts.decision_prompt(user_query, retrivedDocs)
has_answer, response_time = llm.generate_response( query=user_query, 
                                                   retrieved_results=retrivedDocs, 
                                                   prompt_template=decision_prompt,
                                                   model_name=llm.GROQ_MODEL2
                                                 )       

print(f'has_answer: {has_answer}, response time (in millisec): {response_time}')

ModelName: gemma2-9b-it
has_answer: NO, response time (in millisec): 698


In [18]:
# Based on the decision prompt result, 
# we either generate response from context or search online using duckduckgo-search

if has_answer == 'YES':
    print('Context can answer the question, generating response...')
    response_from_context = True
else:
    agent_query = f"site:https://uaelegislation.gov.ae/en/legislations/1594 {user_query}"
    print('Context cannot answer the question, searching online for answer...')
    response_from_context = False
    # response_from_ddgs = DDGS().text(user_query, max_results=5)  # fetching top 5 results from internet
    response_from_ddgs = DDGS().text(agent_query, max_results=10)  # search from specific site
    sorted_docs = re_rank_docs(response_from_ddgs, user_query)
    formatted_results = format_search_results(sorted_docs)
    
retrieved_results = retrivedDocs if response_from_context else formatted_results

# Common code for generating the response
response, response_time = llm.generate_response(
    query=user_query,
    retrieved_results=retrieved_results,
    prompt_template=prompts.response_generation(user_query, retrieved_results),
    model_name=llm.GROQ_MODEL
)

print('Response:')
display(Markdown(response))


Context cannot answer the question, searching online for answer...
1.0 United Arab Emirates Legislations | Federal Law by Decree on...
1.0 UAE Legislation United Arab Emirates Legislations | Legislations
1.0 Federal Law by Decree No. (49) of 2022
ModelName: llama3-70b-8192
Response:


**Types of Recruitment Patterns in UAE**

Based on the provided context, I found the following types of recruitment patterns in UAE:

**1. Federal Government Recruitment Pattern**

* This type of recruitment is governed by the UAE Federal Government laws and regulations.
* It involves recruiting employees for federal government entities, such as ministries and government departments.

**2. Emiratization Recruitment Pattern**

* This type of recruitment is focused on hiring UAE nationals for private sector jobs.
* It aims to increase the number of Emiratis employed in the private sector.

**3. Public-Private Partnership Recruitment Pattern**

* This type of recruitment involves a partnership between public and private sector entities.
* It aims to provide job opportunities for UAE nationals and promote economic growth.

**4. General Recruitment Pattern**

* This type of recruitment is not specific to any particular sector or entity.
* It involves recruiting employees for various job roles across different industries.

Please note that these types of recruitment patterns are based on the provided context and may not be exhaustive.

#### Observation
- duckduckgo-search for a specific website is a hit & miss
- some websites block agent crawlers to index data from their website
- more advanced techniques needs to be explored to make it work